In [2]:
#@title On Time and PPM model v2 (press Play to run)
#Draft last saved 03 Mar 6:22pm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from subprocess import check_output
from datetime import time
#print(check_output(["ls", "../input"]).decode("utf8"))


#noon = time_to_seconds(time(12, 0, 0))
#df.timestamp = df.timestamp.apply(lambda t: abs(noon - t))

# one hot encode categorical columns
#columns = ["Geography Description",]
#df = pd.get_dummies(df, columns=columns)
#df.head(10)


df = pd.read_csv(r"C:\Users\Tfarhy\OneDrive - Network Rail\Profile\Desktop\ppm-on-time-new-data.csv")
df = df.drop(axis=1, labels = ["Date"])
df = df[['Primary Delay per 100 miles','Footfall','Count of Trains/Timing Points - WTT','Planned','On Time WTT%','PPM%']]

# Extract the training and test data
def train():
    global df, labelname, model, model2, submodels, X, y, X_train, X_test, y_train, y_test, scaler, label_df, temp_data, temp_labels, temp_scaler, temp_model, temp_df
    
    df = pd.read_csv(r"C:\Users\Tfarhy\OneDrive - Network Rail\Profile\Desktop\ppm-on-time-new-data.csv")
    df = df.drop(axis=1, labels = ["Date"])
    df = df[['Primary Delay per 100 miles','Footfall','Count of Trains/Timing Points - WTT','Planned','On Time WTT%','PPM%']]
    
    output_columns = list(df.columns[-1:-3:-1])
    
    for i, input_variable in enumerate(df.columns[:-2]):
        if not checkboxes.children[i].value:
            df.pop(input_variable)    
    
    labels = df.pop(buttons.value)
    output_columns.remove(buttons.value)
    
    for unused_output in output_columns:
        df.pop(unused_output)
    
    labelname = labels.name

    #print(df.head())
    #print(labels.head())


    data = df.values
    labels = labels.values

    X = data  # all rows, no label
    y = labels  # all rows, label only
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    X_unscaled = X_train
    
    # Scale the data to be between -1 and 1
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Establish model
    model = RandomForestRegressor(n_jobs=-1, verbose = 0)
    model2 = GradientBoostingRegressor(verbose = 0)
    
    #Make submodels for sticky inference sliders
    
    from sklearn.multioutput import MultiOutputRegressor
    submodels = {}
    for input_variable in df.columns:
        temp_df = df.copy(deep=True)
        temp_data = temp_df.pop(input_variable)
        
        temp_data = temp_data.values.reshape(-1,1)
        temp_labels = temp_df.values
        
        temp_scaler = StandardScaler()
        #temp_scaler.fit(temp_data)
        #temp_data = temp_scaler.transform(temp_data)
        
        temp_model = MultiOutputRegressor(GradientBoostingRegressor(verbose = 0))
        #temp_model.set_params(n_estimators=7)
        temp_model.fit(temp_data, temp_labels)
        
        submodels[input_variable] = (temp_model,temp_scaler)

    # Try different numbers of n_estimators - this will take a minute or so
    if False:
        estimators = np.arange(1, 500, 10)
        scores = []
        for n in estimators:
            model.set_params(n_estimators=n)
            model.fit(X_train, y_train)
            scores.append(model.score(X_test, y_test))
        plt.title("Effect of n_estimators")
        plt.xlabel("n_estimator")
        plt.ylabel("score")
        plt.plot(estimators, scores)

    model.set_params(n_estimators=70)
    model.fit(X_train, y_train)
    model2.fit(X_unscaled, y_train)
    
    
    return model

'''
Cross-validate our model using k-fold cross validation.
'''
def train_and_validate(b):
    global X, y, mae
    print('Generating model...')
    train()
    
    from sklearn.model_selection import RepeatedKFold
    from sklearn.model_selection import cross_val_score
    from numpy import absolute
    from numpy import mean
    from numpy import std

    # define the evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, verbose = 0)
    # force the scores to be positive
    n_scores = absolute(n_scores)
    # summarize performance
    mae = mean(n_scores)
    
    #print(model)
    print(f'Containing variables {list(df.columns)} \nTargeting {labelname} as output')
    print(f'Cross-validated mean absolute error for model: {mae*100:.2f}%, standard deviation {std(n_scores):.2f}')
    
    inference_widgets()

def infer(b, printout = True, whichmodel =['RF','GBM']):
    
    prediction = np.nan
    prediction2 = np.nan
    
    if 'RF' in whichmodel:
        prediction = model.predict(scaler.transform(np.array([box.children[1].value for box in inwidgets.children], ndmin = 2)))[0]
    
    if 'GBM' in whichmodel:
        prediction2 = model2.predict(np.array([box.children[1].value for box in inwidgets.children], ndmin = 2))[0]
    
    if printout:
        print([box.children[1].value for box in inwidgets.children])
        print(f'{labelname} RF = {prediction*100:.2f}%, GBR = {prediction2*100:.2f}% ')
        #print(f'Predicted a {labelname} of {prediction*100:.2f}%.')
    
    else:
        return {'RF':prediction, 'GBM':prediction2 }
    
    

def donothing(*args):
    pass

import sys
from copy import copy

def SliderChanged(change):
    #global p
    
    #pdb.set_trace()
    if synchrobox.value:
        mover = change['owner']

        barewidgets2 = copy(barewidgets)
        barewidgets2.remove(mover)
        
        #k = barewidgets.index(mover)
        
        for barewidget in barewidgets2:
            if barewidget!= mover:
                barewidget.unobserve(SliderChanged, names = 'value')


        i = barewidgets.index(mover)
        name = inwidgets.children[i].children[2].value

        p = submodels[name][0].predict(np.array(mover.value, ndmin = 2))
        
        inference = infer(None, printout = False, whichmodel = 'GBM')
        #print(f"\r{change}", end="")
        print(f"\r{inference['RF']*100:.2f}%, {inference['GBM']*100:.2f}%", end="")

        for j, barewidget in enumerate(barewidgets2):
            if barewidget != mover:
                barepred = p[0][j]
                barechange = barepred - barewidget.value

                #if barepred >= barewidget.value:
                #if barechange<barewidget.max*0.1:
                barewidget.value = barewidget.value + barechange/8

                #elif barepred<barewidget.value:
                #    barewidget.value = barewidget.value - (barewidget.value -barepred)/4



        for barewidget in barewidgets2:
            if barewidget!= mover:
                barewidget.observe(SliderChanged, names = 'value')
    else:
      mover = change['owner']
      for i in barewidgets:
        if i != mover:
          i.value = (mover.value/mover.max)*i.max

from copy import copy
import sys

def SliderChangedFuncGenerator(slider):
    def wrapper(*args):
        owner = slider
        for i, barewidget in enumerate(barewidgets):
            if barewidget is not slider:
                
                name = inwidgets.children[i].children[2].value
                
                inference_vector = [box.children[1].value for box in inwidgets.children]
                inference_vector.pop(i)
                
                barewidget.value = submodels[name][0].predict(submodels[name][1].transform(np.array(inference_vector, ndmin = 2)))[0]
                #sys.stdout.flush()
                print(args, x_widget.value, y_widget.value,  end="\r", flush=False,)
                print(f'i={i}')
    
    return wrapper

#Jupyter widgets

from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout, Label, HBox, VBox

label_layout = Layout(width='150px',height='21px', font_size = '50px')
z = Label('Using these variables:',layout=label_layout)


checkboxes = []

for column in df.columns[:-2]:
    checkbox = widgets.Checkbox(
        value=True,
        description=column,
        disabled=False,
        indent=True,
        layout=Layout(height='18px', padding = '1px')
    )
    checkboxes.append(checkbox)
    
checkboxes = VBox(checkboxes)    


buttons = widgets.ToggleButtons(
    options=df.columns[-1:-3:-1],
    description='Predict what:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or '',
#     icons=['check'] * 3
)


def inference_widgets():
    global inwidgets, inferbutton, barewidgets, synchrobox, bareboxes
    
    inwidgets = []
    
    for i, input_variable in enumerate(df.columns):
        
        if input_variable == 'Primary Delay per 100 miles':
            w =HBox([widgets.IntSlider(min = 0, max = df[input_variable].max()*1.2, value = int(df[58:59][input_variable]), layout=Layout(width='200px'), readout = False,  continuous_update=True, step = 1),
                     widgets.FloatText(value = int(df[58:59][input_variable]), layout=Layout(width='120px', padding = '0px 10px 0px 10px')  ),
                     Label(input_variable),])
            
        else:
            w =HBox([widgets.IntSlider(min = 0, max = df[input_variable].max()*1.2, value = int(df[58:59][input_variable]), layout=Layout(width='200px'), readout = False,  continuous_update=True), 
                     widgets.IntText(value = int(df[58:59][input_variable]), layout=Layout(width='120px', padding = '0px 10px 0px 10px')  ),
                     Label(input_variable),])
        
        widgets.link((w.children[0], 'value'), (w.children[1], 'value'))
        
        w.children[0].observe(SliderChanged, 'value')
        inwidgets.append(w)
    
    inwidgets = VBox(inwidgets)
    barewidgets = [hbox.children[0] for hbox in inwidgets.children]
    bareboxes = [hbox.children[1] for hbox in inwidgets.children]
    
    #for w in inwidgets.children:
        #SliderChangedFunc = SliderChangedFuncGenerator(w.children[0])
        #w.children[0].observe(SliderChangedFunc, names = 'value')
    
    
    
    inferbutton = widgets.Button(description=f"Predict {labelname}", layout=Layout(width='200px'))
    
    synchrobox = widgets.Checkbox(
        value=True,
        description='Synchronise sliders',
        disabled=False,
        indent=False,
        layout=Layout(height='18px', padding = '1px')
    )
    
    
    inferbutton.on_click(infer)
    
    
    display(inwidgets)
    display(synchrobox)
    display(inferbutton)
    
    

button = widgets.Button(description="Generate model")

display(z)
display(checkboxes)
display(buttons)
display(button)

button.on_click(train_and_validate)

#train_and_validate(None)

#savestamp

Label(value='Using these variables:', layout=Layout(height='21px', width='150px'))

ToggleButtons(description='Predict what:', options=('PPM%', 'On Time WTT%'), value='PPM%')

Button(description='Generate model', style=ButtonStyle())

In [1]:
print(x)

NameError: name 'x' is not defined